In [1]:
import os
import datetime
import pandas as pd
from pandas import json_normalize
import numpy as np
import re
import json
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random
import pprint
from string import punctuation
from wordcloud import WordCloud 

/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/viviando/.local/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
/var/folders/b8/4ntn3_wd1wg59r0lmbfgfmwc0000gn/T/ipykernel_34604/3852159387.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import panda

In [4]:
# define functions

punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation 


def descriptive_stats(tokens, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity, and num_tokens most common
        tokens. Return a list of 
    """
    num_tokens=len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = num_unique_tokens/num_tokens
    num_characters = sum(len(token) for token in tokens)
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        counter = Counter(tokens)
        top_5_tokens = counter.most_common(5)
        print("Top 5 most common tokens:")
        for token, count in top_5_tokens:
            print(f"{token}: {count} occurrences")
            
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
def remove_stop(tokens) :
    return [token for token in tokens if token not in sw]
    return(tokens)
 
def remove_punctuation(text, punct_set=tw_punct) : 
    """
        Function takes two arguments: (1) text, which is the input string, and (2) the punctuation set, which is set to the tw_punct value set.
        Returns all characters not found in the punctuation set and concatenates them back into a string using the .join() method with an empty
        string "" as the separator. 
    """
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text) : 
    """ 
        Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. 
    """
    tokens = text.split()
    return(tokens)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

In [63]:
from matplotlib import pyplot as plt

def wordcloud(word_freq, title=None, max_words=200, stopwords=None):

    wc = WordCloud(width=800, height=400, 
                   background_color= "black", colormap="Paired", 
                   max_font_size=150, max_words=max_words)
    
    # convert data frame into dict
    if type(word_freq) == pd.Series:
        counter = Counter(word_freq.fillna(0).to_dict())
    else:
        counter = word_freq

    # filter stop words in frequency counter
    if stopwords is not None:
        counter = {token:freq for (token, freq) in counter.items() 
                              if token not in stopwords}
    wc.generate_from_frequencies(counter)
 
    plt.title(title) 

    plt.imshow(wc, interpolation='bilinear')
    plt.axis("off")
    
    
def count_words(df, column='titles_tokens', preprocess=None, min_freq=2):

    # process tokens and update counter
    def update(doc):
        tokens = doc if preprocess is None else preprocess(doc)
        counter.update(tokens)

    # create counter and run through all data
    counter = Counter()
    df[column].map(update)

    # transform counter into data frame
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    freq_df = freq_df.query('freq >= @min_freq')
    freq_df.index.name = 'token'
    
    return freq_df.sort_values('freq', ascending=False)

In [51]:
# data import
recipes = pd.read_csv('recipes.csv')
#recipes=pd.DataFrame(recipes)
#recipes['title']=recipes['title'].astype(str)
#recipes['ingredients']=recipes['ingredients'].astype(str)
#recipes['step']=recipes['step'].astype(str)
#recipes.dtypes

In [48]:
recipes

,title,ingredients,step
0,Cannellini Bean and Asparagus Salad with Mushr...,"cannellini beans, water, water, curry leaves, ...",Rinse the cannellini beans and soak for 8 hour...
1,Berry Banana Breakfast Smoothie,"yogurt, graham cracker crumbs, soymilk, berrie...",Take some yogurt in your favorite flavor and a...
2,Red Lentil Soup with Chicken and Turnips,"olive oil, soup, carrot, celery, onion, salt a...","To a large dutch oven or soup pot, heat the ol..."
3,Asparagus and Pea Soup: Real Convenience Food,"garlic, onion, garlic, onion, extra virgin oli...",Chop the garlic and onions. Saute the onions i...
4,Garlicky Kale,"olive oil, kale, vinegar, garlic",Heat the olive oil in a large pot over medium ...
5,Slow Cooker Beef Stew,"cream of mushroom soup, beef broth, seasoning,...","To get started, heat your slow cooker to low. ..."
6,Red Kidney Bean Jambalaya,"kidney beans, brown rice, water, kidney beans,...",Rinse the kidney beans and brown rice separate...
7,Chicken Fajita Stuffed Bell Pepper,"chili powder, cilantro, pepper, quinoa, cumin,...","To get started heat oven to 35 Mix salt, peppe..."
8,"Cauliflower, Brown Rice, and Vegetable Fried Rice","cauliflower florets, cauliflower rice, caulifl...",Remove the cauliflower's tough stem and reserv...
9,Hummus and Za'atar,"chickpeas, water, water, canned chickpeas, pin...",Rinse the chickpeas and soak for 8 hours or ov...


In [67]:
# store tokens in new dataframe 'df'
df=pd.DataFrame()

# fold to lowercase
df['titles_tokens']=recipes['title'].str.lower()
df['ingredients_tokens']=recipes['ingredients'].str.lower()
df['steps_tokens']=recipes['step'].str.lower()

# remove punctuation
df['ingredients_tokens']=df['ingredients_tokens'].apply(remove_punctuation)
df['steps_tokens']=df['steps_tokens'].apply(remove_punctuation)
df['titles_tokens']=df['titles_tokens'].apply(remove_punctuation)

# tokenize
df['titles_tokens']=tokenize(df['titles_tokens'].str)
df['ingredients_tokens']=tokenize(df['ingredients_tokens'].str)
df['steps_tokens']=tokenize(df['steps_tokens'].str)

df

,titles_tokens,ingredients_tokens,steps_tokens
0,"[cannellini, bean, and, asparagus, salad, with...","[cannellini, beans, water, water, curry, leave...","[rinse, the, cannellini, beans, and, soak, for..."
1,"[berry, banana, breakfast, smoothie]","[yogurt, graham, cracker, crumbs, soymilk, ber...","[take, some, yogurt, in, your, favorite, flavo..."
2,"[red, lentil, soup, with, chicken, and, turnips]","[olive, oil, soup, carrot, celery, onion, salt...","[to, a, large, dutch, oven, or, soup, pot, hea..."
3,"[asparagus, and, pea, soup, real, convenience,...","[garlic, onion, garlic, onion, extra, virgin, ...","[chop, the, garlic, and, onions, saute, the, o..."
4,"[garlicky, kale]","[olive, oil, kale, vinegar, garlic]","[heat, the, olive, oil, in, a, large, pot, ove..."
5,"[slow, cooker, beef, stew]","[cream, of, mushroom, soup, beef, broth, seaso...","[to, get, started, heat, your, slow, cooker, t..."
6,"[red, kidney, bean, jambalaya]","[kidney, beans, brown, rice, water, kidney, be...","[rinse, the, kidney, beans, and, brown, rice, ..."
7,"[chicken, fajita, stuffed, bell, pepper]","[chili, powder, cilantro, pepper, quinoa, cumi...","[to, get, started, heat, oven, to, 35, mix, sa..."
8,"[cauliflower, brown, rice, and, vegetable, fri...","[cauliflower, florets, cauliflower, rice, caul...","[remove, the, cauliflowers, tough, stem, and, ..."
9,"[hummus, and, zaatar]","[chickpeas, water, water, canned, chickpeas, p...","[rinse, the, chickpeas, and, soak, for, 8, hou..."


In [73]:
title_combined_tokens = [token for sublist in df['titles_tokens'] for token in sublist]
#descriptive_stats(sza_combined_tokens)

descriptive_stats(title_combined_tokens)

There are 50 tokens in the data.
There are 39 unique tokens in the data.
There are 282 characters in the data.
The lexical diversity is 0.780 in the data.
Top 5 most common tokens:
and: 5 occurrences
bean: 2 occurrences
asparagus: 2 occurrences
with: 2 occurrences
red: 2 occurrences


[50, 39, 0.78, 282]

In [74]:
len(df)

10